In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing, linear_model

In [35]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [36]:
y = train.target.values
train = train.drop(['ID', 'target'], axis=1)
test = test.drop('ID', axis=1)

In [55]:
train

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739e+00,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,3.575369e-02,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144e+00
1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,...,NaN,NaN,5.988956e-01,AF,NaN,NaN,1.957825,0,NaN,NaN
2,9.438769e-01,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,1.345191e-02,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472e+00
3,7.974146e-01,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,2.267384e-03,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483e+00
4,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN
5,NaN,NaN,C,NaN,8.856791,NaN,NaN,0.359993,NaN,1.050328,...,NaN,NaN,4.986116e-02,X,NaN,NaN,1.536222,0,NaN,NaN
6,8.998057e-01,7.312995,C,3.494148,9.946200,1.926070,1.770427,0.066251,5.011287,2.341356,...,3.476299,1.992594,8.375832e-02,BJ,3.276100,1.623298,2.266575,0,2.263736,9.708730e-01
7,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.838074,...,NaN,NaN,NaN,BY,NaN,NaN,NaN,0,NaN,NaN
8,2.078651e+00,8.462619,NaN,3.739030,5.265636,1.573033,2.303371,0.015869,11.111111,4.463894,...,8.148148,1.875560,1.865950e-02,S,1.159637,5.582865,1.105283,0,1.170731,3.333334e+00
9,1.144802e+00,5.880606,C,3.244469,9.538384,2.500001,1.559405,0.412610,9.977529,2.363238,...,7.325843,4.896617,8.943653e-03,E,1.344550,1.601176,1.928009,0,3.174603,1.000000e+00


In [5]:
train.shape

(114321, 131)

In [58]:
train2 = train.dropna(thresh=70)

In [59]:
train2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739e+00,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000e+00,1.989780,3.575369e-02,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144e+00
2,9.438769e-01,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333e+00,2.477596,1.345191e-02,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472e+00
3,7.974146e-01,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256e+00,1.812795,2.267384e-03,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483e+00
6,8.998057e-01,7.312995,C,3.494148,9.946200,1.926070,1.770427,0.066251,5.011287,2.341356,...,3.476299e+00,1.992594,8.375832e-02,BJ,3.276100,1.623298,2.266575,0,2.263736,9.708730e-01
8,2.078651e+00,8.462619,NaN,3.739030,5.265636,1.573033,2.303371,0.015869,11.111111,4.463894,...,8.148148e+00,1.875560,1.865950e-02,S,1.159637,5.582865,1.105283,0,1.170731,3.333334e+00
9,1.144802e+00,5.880606,C,3.244469,9.538384,2.500001,1.559405,0.412610,9.977529,2.363238,...,7.325843e+00,4.896617,8.943653e-03,E,1.344550,1.601176,1.928009,0,3.174603,1.000000e+00
13,1.400267e+00,5.367204,C,4.122155,8.137188,2.983080,2.640249,0.211851,11.021506,1.816193,...,8.817203e+00,2.096062,5.459061e-07,AQ,1.731656,6.102516,1.388117,4,1.220911,1.878453e+00
14,2.260036e+00,14.693263,C,5.150750,8.554136,1.954626,2.931936,0.041446,7.022901,1.291029,...,5.038168e+00,1.836667,5.973979e-03,Z,3.242128,0.818063,2.400050,0,1.166666,2.857143e+00
16,6.228961e-01,7.024732,C,4.193688,6.288177,2.132436,3.198654,0.407525,10.434782,1.312909,...,6.086957e+00,2.812624,2.474223e-01,BJ,1.865530,3.345960,0.796533,0,1.389474,6.060615e-01
20,9.438780e-01,5.927194,C,4.404372,9.045057,2.551021,2.693878,0.085665,8.601036,3.916848,...,6.632125e+00,2.419037,1.847926e-02,U,1.884567,2.602041,1.959776,1,1.121212,1.351351e+00


In [ ]:
train = train.dropna(axis=1, thresh=2000)
test = test.dropna(axis=1, thresh=2000)